In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf

ModuleNotFoundError: No module named 'pandas'

In [ ]:
dataset=pd.read_csv("sdn_upda2.csv",index_col='TimeStamp',parse_dates=True)
dataset

In [ ]:
dataset=dataset.drop(['IP Address','s4','Priority','s2','s3','s5','Membership'],axis=1)
dataset

In [ ]:
sc=MinMaxScaler(feature_range=(0,1))
dataset=sc.fit_transform(dataset)
dataset

In [ ]:
Train_x=[]
Train_y=[]
for i in range(50,6048):
   Train_x.append(dataset[i-50:i,0])
   Train_y.append(dataset[i])

Train_y

In [ ]:
Train_x=np.array(Train_x)
Train_y=np.array(Train_y)
Train_x.shape


In [ ]:

Train_x=np.reshape(Train_x,(Train_x.shape[0],Train_x.shape[1],1))
Train_x.shape

In [ ]:
unix=tf.keras.Sequential()
unix.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units=100, return_sequences=True,input_shape=(Train_x.shape[1],1))))
unix.add(tf.keras.layers.Dropout(0.2))

unix.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units=100,return_sequences=True)))
unix.add(tf.keras.layers.Dropout(0.2))

unix.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units=50)))
unix.add(tf.keras.layers.Dropout(0.2))

unix.add(tf.keras.layers.Dense(units=1,activation='relu'))

In [ ]:
unix.compile(optimizer='adam',loss='mean_squared_error',metrics=['mae'])

In [ ]:
BILSTM=unix.fit(Train_x,Train_y,epochs=15)


In [ ]:
import pickle
from keras.models import model_from_json

In [ ]:
pickle.dump(sc, open('../S1_upda2/unixscaler_BL5.pkl','wb'))

In [ ]:
model_jsk = unix.to_json()
with open("../S1_upda2/Traffic_model_BL5.json", "w") as source:
    source.write(model_jsk)
# save model parameter
unix.save_weights("../S1_upda2/Traffic_model_BL5.h5")

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
sc= pickle.load(open('../S1_upda2/unixscaler_BL5.pkl','rb'))

with open("../S1_upda2/Traffic_model_BL5.json", "r") as f:
    unix = model_from_json(f.read())
unix.load_weights("../S1_upda2/Traffic_model_BL5.h5")

In [ ]:
dataset=pd.read_csv("sdn_upda2.csv",index_col='TimeStamp',parse_dates=True)
dataset=dataset.drop(['IP Address','s4','Priority','s2','s3','s5','Membership'],axis=1)
copy=pd.read_csv("sdn_upda2.csv",index_col='TimeStamp',parse_dates=True)
copy=copy.drop(['IP Address','s4','Priority','s2','s3','s5','Membership'],axis=1)
copy=copy.values



with open("s1", "rb") as fp:   # Unpickling
 sarima = pickle.load(fp)


dataset=sc.fit_transform(dataset)
real_unixred=dataset[6049:,0]
real_unix=copy[6049:,0]
x_test=[]
for i in range(6048,8064):
    x_test.append(dataset[i-50:i,0])

X_test=np.array(x_test)

X_test=np.reshape(X_test,(X_test.shape[0],X_test.shape[1],1))
predicted_unixred=unix.predict(X_test)

predicted_unix=sc.inverse_transform(predicted_unixred)
print(predicted_unix)


with open("bilstm", "wb") as source:
    pickle.dump(predicted_unix,source)


fig = plt.figure()
ax = fig.add_axes([0.1, 0.1, 0.8, 0.8]) 
y = sarima
x = [i for i in range(0, 288)]

ax.plot(x,predicted_unix[:288],color="red",label='Bilstm Prediction')
ax.plot(x,y[:288],color="yellow",label='Sarima Prediction')
ax.plot(x,real_unix[:288],color="green",label='Real')
ax.set_xlabel('Hours in predicted Day')
ax.set_ylabel('Traffic in Gigabytes')
ax.set_title('Prediction Comparision')
ax.set_xticks([i for i in range(0,289,24)])
ax.legend()
ax.set_xticklabels([j for j in range(0,25,2)])
# plt.axvspan(1446, 1451, facecolor='r', alpha=0.5)
plt.show()


In [ ]:
from sklearn.metrics import mean_squared_error,mean_absolute_error


print("MEAN SQUARE ERROR")
print(np.sqrt(mean_squared_error(real_unixred,predicted_unixred[:-1])))
print()
print("MEAN ABSOLUTE ERROR")
print(mean_absolute_error(real_unixred,predicted_unixred[:-1]))

mape = np.mean(np.abs(real_unix-predicted_unix)/real_unix)*100
mape


: 

In [ ]:
# with open("predweek2upda", "wb") as fp:   #Pickling
#    pickle.dump(prdiction_Switch, fp)


# import pickle
# with open("predweek2upda", "rb") as fp:   # Unpickling
#  jsk = pickle.load(fp)
